In [14]:
print("ok")

ok


In [15]:
%pwd

'c:\\Users\\lokes\\Medichat\\research'

In [16]:
import os
os.chdir("../")

In [17]:
%pwd

'c:\\Users\\lokes\\Medichat'

In [18]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [19]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    
    documents=loader.load()
    
    return documents

In [20]:
extracted_data=load_pdf_file(data='Data/')

In [21]:
# extracted_data

In [22]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [23]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [24]:
# text_chunks

In [25]:
from langchain.embeddings import HuggingFaceEmbeddings

In [26]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [27]:
embeddings=download_hugging_face_embeddings()

C:\Users\lokes\AppData\Local\Temp\ipykernel_16344\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [28]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [29]:
#query_result

In [30]:
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [32]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "mbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [33]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [34]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [35]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [36]:
docsearch

In [37]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [38]:
retrieved_docs = retriever.invoke("What is Acne?")

In [39]:
retrieved_docs

[Document(id='34a89627-66ca-43ca-a869-213ada7a50e8', metadata={'page': 39.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='a7b7a251-88c4-414b-ac5c-b04f38a4a833', metadata={'page': 38.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='14bb833b-3b52-434c-94ac-8a6a399272a7', metadata={'page': 37.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when

In [40]:
import ollama
from IPython.display import Markdown, display
prompt="What is pandas dataframe?" 
response= ollama.chat(model='llama3.2', messages=[{'role':'user','content':prompt}])
response_content = response['message']['content']
display(Markdown(f"**LLM Response:**\n\n{response_content}"))

**LLM Response:**

**Pandas DataFrames**

A Pandas DataFrame is a two-dimensional table of data with rows and columns, similar to an Excel spreadsheet or a SQL database table. It's a fundamental data structure in the Pandas library, which is widely used for data manipulation and analysis in Python.

**Key Characteristics:**

1. **Tabular structure**: DataFrames are composed of rows and columns, making it easy to visualize and manipulate data.
2. **Data types**: Each column can have a different data type, such as integers, floats, strings, or dates.
3. **Flexible data**: DataFrames can store various data types, including nested structures like lists or dictionaries.

**Example Use Cases:**

1. **Data analysis**: Load and manipulate datasets for data analysis, visualization, and reporting.
2. **Data cleaning and preprocessing**: Clean and preprocess data by handling missing values, converting data types, and merging datasets.
3. **Data storage and retrieval**: Store and retrieve data efficiently using DataFrames.

**Basic Operations:**

1. **Creating a DataFrame**: Use the `pd.DataFrame()` constructor or the `pd.read_csv()` function to create a new DataFrame from a CSV file or other data sources.
2. **Selecting data**: Use indexing, slicing, or label-based selection to extract specific rows and columns.
3. **Filtering data**: Apply conditional statements using boolean operations (e.g., `df[df['age'] > 30]`) to filter rows based on conditions.

**Example Code:**
```python
import pandas as pd

# Create a sample DataFrame
data = {'Name': ['John', 'Mary', 'David'],
        'Age': [25, 31, 42],
        'Country': ['USA', 'UK', 'Canada']}
df = pd.DataFrame(data)

print(df)
```
Output:
```
     Name  Age    Country
0    John   25       USA
1    Mary   31         UK
2   David   42    Canada
```
In this example, we create a simple DataFrame with three columns and three rows. We can then manipulate the data using various Pandas functions and methods.

**Tips and Tricks:**

* Use `df.head()` to display the first few rows of a DataFrame.
* Use `df.info()` to get an overview of the DataFrame's structure and memory usage.
* Use `df.describe()` to calculate summary statistics for numeric columns.

In [41]:
import ollama
from IPython.display import display,Markdown,clear_output
prompt = "what is Acne?"
response_stream= ollama.chat(
    model='llama3.2',
    messages = [{'role':'user', 'content': prompt}],
    stream=True  
)

streamed_response = ""
for token in response_stream:
    streamed_response += token['message']['content']
    clear_output(wait=True)
    display(Markdown(f"**LLM Response (Streaming ):**\n\n{streamed_response}"))


**LLM Response (Streaming ):**

Acne is a skin condition characterized by the occurrence of comedones (blackheads and whiteheads), pimples, cysts, and inflammatory lesions on the skin. It affects both children and adults, with the peak age range being adolescence to early adulthood.

Causes of Acne:

1. **Hormonal changes**: Fluctuations in hormone levels, particularly androgens (male hormones), can lead to increased sebum production, clogged pores, and inflammation.
2. **Bacteria**: A type of bacteria called Propionibacterium acnes (P. acnes) is naturally present on the skin and can contribute to acne.
3. **Clogged pores**: Dead skin cells, oil, and other debris can combine to form a plug in the pore, leading to clogged pores and acne.
4. **Genetics**: Family history and genetic predisposition can play a role in the development of acne.
5. **Stress**: Stress can increase hormone production and lead to inflammation, which can exacerbate acne.

Symptoms of Acne:

1. **Comedones** (blackheads and whiteheads): Small bumps on the skin that are usually black or white in color.
2. **Pimples**: Red, inflamed bumps on the skin that can be painful and tender.
3. **Cysts**: Large, painful bumps under the skin that may leave scars after they heal.
4. **Inflammatory lesions**: Red, swollen areas on the skin that can be painful and itchy.

Types of Acne:

1. **Primary acne**: The most common type of acne, characterized by comedones and pimples.
2. **Secondary acne**: Acne that develops in response to other medical conditions, such as hormonal imbalances or inflammatory disorders.
3. **Keratosis pilaris**: A condition characterized by small, rough bumps on the skin, often associated with acne.

Treatment of Acne:

1. **Topical treatments**: Creams and gels containing salicylic acid, benzoyl peroxide, or sulfur can help to unclog pores and reduce inflammation.
2. **Oral medications**: Antibiotics, retinoids, and hormonal therapies may be prescribed to manage acne.
3. **Lifestyle changes**: Maintaining a healthy diet, staying hydrated, exercising regularly, and managing stress can help to prevent acne.

If you're experiencing acne, it's essential to consult with a dermatologist or healthcare professional for proper diagnosis and treatment.